![HSV-AI Logo](https://github.com/HSV-AI/hugo-website/blob/master/static/images/logo_v9.png?raw=true)

# Implicit Recommendation from ECommerce Data

Some of the material for this work is based on [A Gentle Introduction to Recommender Systems with Implicit Feedback](https://jessesw.com/Rec-System/) by Jesse Steinweg Woods. This tutorial includes an implementation of the Alternating Least Squares algorithm and some other useful functions (like the area under the curve calculation). Other parts of the tutorial are based on a previous version of the Implicit library and had to be reworked.

The dataset used for this work is from Kaggle [Vipin Kumar Transaction Data](https://www.kaggle.com/vipin20/transaction-data):

## Context

This is a item purchased transactions data. It has 8 columns.
This data makes you familer with transactions data.

## Content

Data description is :-

* UserId -It is a unique ID for all User Id
* TransactionId -It contains unique Transactions ID
* TransactionTime -It contains Transaction Time
* ItemCode -It contains item code that item will be purchased
* ItemDescription -It contains Item description
* NumberOfItemPurchased -It contains total number of items Purchased
* CostPerltem -Cost per item Purchased
* Country -Country where item purchased


# Global Imports

In [1]:
import pandas as pd
import numpy as np
import random
from matplotlib import pyplot as plt
import implicit
import scipy
from sklearn import metrics
from pandas.api.types import CategoricalDtype

import wandb

In [2]:
run = wandb.init(project="vipin20-implicit", reinit=True)

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ········


wandb: Appending key for api.wandb.ai to your netrc file: /home/vscode/.netrc
wandb: wandb version 0.11.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [3]:
transactions = pd.read_pickle('../data/interim/vipin20/transactions.gz')
print('Loaded',len(transactions),'rows')

Loaded 1083818 rows


In [4]:
transaction_list = list(np.sort(transactions.TransactionId.unique())) # Get our unique customers
item_list = list(transactions.ItemCode.unique()) # Get our unique products that were purchased
quantity_list = list(transactions.NumberOfItemsPurchased) # All of our purchases

cols = transactions.TransactionId.astype(CategoricalDtype(categories=transaction_list, ordered=True)).cat.codes 
# Get the associated row indices
rows = transactions.ItemCode.astype(CategoricalDtype(categories=item_list, ordered=True)).cat.codes 
# Get the associated column indices
purchases_sparse = scipy.sparse.csr_matrix((quantity_list, (rows, cols)), shape=(len(item_list), len(transaction_list)))

In [5]:
matrix_size = purchases_sparse.shape[0]*purchases_sparse.shape[1] # Number of possible interactions in the matrix
num_purchases = len(purchases_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

99.41153632142772

# Training & Test Datasets

We will use the function below to create a training and test dataset from the tutorial linked at the top. The test dataset masks some percentage of purchases to tested later with a recommendation.

In [6]:
def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)), list(set(item_inds)) # Output the unique list of user rows that were altered  


In [ ]:
wandb.log({'percent_test':0.211})

In [125]:
product_train, product_test, products_altered, transactions_altered = make_train(purchases_sparse, pct_test = 0.211)
print('Total number of masked items:',product_test.count_nonzero()-product_train.count_nonzero())


Total number of masked items: 51927


# Implicit Recommendation Model

The code below creates and trains one of the models available from the Implicit package. Currently using hyperparameters suggested by various tutorials with no tuning.

In [126]:
alpha = 3
factors = 64
regularization = 0.003
iterations = 84

model = implicit.als.AlternatingLeastSquares(factors=factors,
                                    regularization=regularization,
                                    iterations=iterations)

## BayesianPersonalizedRanking was pretty bad
# model = implicit.bpr.BayesianPersonalizedRanking(factors=31,
#                                     regularization=0.1,
#                                     iterations=50)


# model = implicit.lmf.LogisticMatrixFactorization(factors=32,
#                                     regularization=0.1,
#                                     iterations=50)

model.fit((product_train * alpha).astype('double'))

user_vecs = model.user_factors
item_vecs = model.item_factors

# Deprecated function below
# user_vecs, item_vecs = implicit.alternating_least_squares((product_train*alpha).astype('double'), 
#                                                           factors=32, 
#                                                           regularization = 0.1, 
#                                                           iterations = 50)

In [70]:
np.save('../data/interim/vipin20/user_factors', user_vecs)
np.save('../data/interim/vipin20/item_factors', item_vecs)
np.save('../data/interim/vipin20/product_train', product_train*alpha)

In [ ]:
wandb.log({
    'alpha': alpha,
    'factors': factors,
    'regularization': regularization,
    'iterations': iterations
})

# Scoring the Model

Following the tutorial, we will use the area under the Receiver Operating Characteristic curve. 

In [8]:
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)   


In [9]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return np.mean(store_auc), np.mean(popularity_auc)  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark


In [137]:
test, popular = calc_mean_auc(product_train, products_altered, 
              [scipy.sparse.csr_matrix(item_vecs), scipy.sparse.csr_matrix(user_vecs.T)], product_test)


print('Our model scored',test,'versus a score of',popular,'if we always recommended the most popular item.')

Our model scored 0.8809457256970782 versus a score of 0.8276412548744586 if we always recommended the most popular item.


In [ ]:
wandb.log({
    'prediction_auc': test,
    'popular_auc':popular
})
run.finish()


# Spot Checking

Now that we have a pretty good idea of the model performance overall, we can spot check a few things like finding similar items and checking item recommendations for an existing invoice.

In [83]:
item_lookup = transactions[['ItemCode', 'ItemDescription']].drop_duplicates() # Only get unique item/description pairs
item_lookup['ItemCode'] = item_lookup.ItemCode.astype(str) # Encode as strings for future lookup ease

price_lookup = transactions[['ItemCode', 'CostPerItem']].drop_duplicates() # Only get unique item/description pairs
price_lookup['ItemCode'] = price_lookup.ItemCode.astype(str) # Encode as strings for future lookup ease


In [75]:
related = model.similar_items(1284)
for rel in related:
    index = rel[0]
    prob = rel[1]
    item = item_lookup[item_lookup.ItemCode == str(item_list[index])].values
    print(prob, item[0][1])

1.0000002 MIRROR MOSAIC GOBLET CANDLE HOLDER
0.810074 MIRROR MOSAIC HURRICANE LAMP 
0.80939496 OLD ROSE COMBO BEAD NECKLACE
0.7795437 PINK ROSEBUD PEARL EARRINGS
0.7791892 AMBER GLASS/SHELL/PEARL NECKLACE
0.77404785 PINK DAISY BAG CHARM
0.7715294 PINK BUTTERFLY HANDBAG W BOBBLES
0.764769 MIRROR MOSAIC CANDLE PLATE
0.75250345 GLASS AND PAINTED BEADS BRACELET TO
0.75219464 PINK YELLOW PATCH CUSHION COVER


In [76]:
user_items = (product_train * alpha).astype('double').T.tocsr()
def recommend(order):
    print('Order Contents:')
    print(transactions[transactions.TransactionId == transaction_list[order]].loc[:, ['ItemCode', 'ItemDescription']])
    print('Recommendations:')
    recommendations = model.recommend(order, user_items)
    for rec in recommendations:
        index = rec[0]
        prob = rec[1]
        stock_code = item_list[index]
        item = item_lookup[item_lookup.ItemCode == str(item_list[index])].values
        print(prob, stock_code, item[0][1])

In [77]:
recommend(1)

Order Contents:
         ItemCode            ItemDescription
85667      475272  HAND WARMER RED POLKA DOT
625518     475293     HAND WARMER UNION JACK
701952     475293     HAND WARMER UNION JACK
1022129    475272  HAND WARMER RED POLKA DOT
Recommendations:
0.79123724 492219 HAND WARMER RED LOVE HEART
0.7557365 480165 HAND WARMER OWL DESIGN
0.74153143 480186 HAND WARMER SCOTTY DOG DESIGN
0.7353631 480207 HAND WARMER BIRD DESIGN
0.5746369 479514 HAND WARMER BABUSHKA DESIGN
0.37681413 440454 12 PENCILS SMALL TUBE SKULL
0.35907966 464394 HOT WATER BOTTLE TEA AND SYMPATHY
0.3555015 1764609 KNITTED UNION FLAG HOT WATER BOTTLE
0.3400025 451185 RETROSPOT HEART HOT WATER BOTTLE
0.3363659 451101 FAWN BLUE HOT WATER BOTTLE


In [130]:
recommend(24000)

Order Contents:
         ItemCode                      ItemDescription
366134     461517   PACK OF 60 PINK PAISLEY CAKE CASES
463306     489489  SET OF 60 I LOVE LONDON CAKE CASES 
477037     489447  SET OF 60 PANTRY DESIGN CAKE CASES 
507191     489489  SET OF 60 I LOVE LONDON CAKE CASES 
757468     489447  SET OF 60 PANTRY DESIGN CAKE CASES 
814991    1784811          60 TEATIME FAIRY CAKE CASES
928197     461517   PACK OF 60 PINK PAISLEY CAKE CASES
1064015   1784811          60 TEATIME FAIRY CAKE CASES
Recommendations:
1.0112753 489468 SET OF 60 VINTAGE LEAF CAKE CASES 
0.9815397 1784832 72 SWEETHEART FAIRY CAKE CASES
0.93189025 481971 60 CAKE CASES DOLLY GIRL DESIGN
0.9093396 461475 PACK OF 60 DINOSAUR CAKE CASES
0.88376737 470757 PACK OF 60 SPACEBOY CAKE CASES
0.85994697 461496 PACK OF 60 MUSHROOM CAKE CASES
0.7270084 445452 PACK OF 72 RETROSPOT CAKE CASES
0.69083047 489426 SET OF 36 DOILIES PANTRY DESIGN
0.64197147 1784727 SET OF 36 TEATIME PAPER DOILIES
0.63782585 482853 SET OF 

In [135]:
transactions['ItemTotal'] = transactions['NumberOfItemsPurchased'] * transactions['CostPerItem']

In [129]:
recommended_price = []
for user in transactions_altered:
    recommendations = model.recommend(user, user_items)
    index = recommendations[0][0]
    price = price_lookup[price_lookup.ItemCode == str(item_list[index])].values
    item = item_lookup[item_lookup.ItemCode == str(item_list[index])].values
    recommended_price.append(price[0][1])
    
total_recommended = np.sum(recommended_price)

print('After recommending',len(transactions_altered),'items, there would be an increase of',
      "${:,.2f}".format(total_recommended*test),'in additional purchases.')

After recommending 2989 items, there would be an increase of $75,298.72 in additional purchases.


## Hyperparameter Tuning with Weights & Biases


In [11]:
sweep_config = {
    "method": "bayes",  # grid, random
    "metric": {"name": "prediction_auc", "goal": "maximize"},
    "parameters": {
        "percent_test": {"min":0.1, "max":0.3},
        "alpha": {"min":1, "max":50 },
        "regularization": {"min":0.001, "max":.3},
        "iterations": {"min":20, "max":100}
    },
}

sweep_id = wandb.sweep(sweep_config, project="Vipin20 Implicit Sweep")

def sweep():

    # Initialize a new wandb run
    wandb.init()

    transactions = pd.read_pickle('../data/interim/vipin20/transactions.gz')
    
    transaction_list = list(np.sort(transactions.TransactionId.unique())) # Get our unique customers
    item_list = list(transactions.ItemCode.unique()) # Get our unique products that were purchased
    quantity_list = list(transactions.NumberOfItemsPurchased) # All of our purchases

    cols = transactions.TransactionId.astype(CategoricalDtype(categories=transaction_list, ordered=True)).cat.codes 
    # Get the associated row indices
    rows = transactions.ItemCode.astype(CategoricalDtype(categories=item_list, ordered=True)).cat.codes 
    # Get the associated column indices
    purchases_sparse = scipy.sparse.csr_matrix((quantity_list, (rows, cols)), shape=(len(item_list), len(transaction_list)))

    product_train, product_test, products_altered, transactions_altered = make_train(purchases_sparse, pct_test = 0.1)

    model = implicit.als.AlternatingLeastSquares(factors=64,
                                    regularization=wandb.config['regularization'],
                                    iterations=wandb.config['iterations'])

    alpha = wandb.config['alpha']                                                                  
    model.fit((product_train * alpha).astype('double'))

    user_vecs = model.user_factors
    item_vecs = model.item_factors
                                                                      
    test, popular = calc_mean_auc(product_train, products_altered, 
                  [scipy.sparse.csr_matrix(item_vecs), scipy.sparse.csr_matrix(user_vecs.T)], product_test)

    wandb.log({
        'prediction_auc': test
    })

    # Sync wandb
    wandb.join()

wandb.agent(sweep_id, sweep)


Create sweep with ID: ticqipfh
Sweep URL: https://wandb.ai/jperiodlangley/Vipin20%20Implicit%20Sweep/sweeps/ticqipfh


wandb: Agent Starting Run: ibnqxuow with config:
wandb: 	alpha: 21
wandb: 	iterations: 34
wandb: 	percent_test: 0.1249590301737199
wandb: 	regularization: 0.11354105529448272
wandb: wandb version 0.11.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


prediction_auc,0.88179
_runtime,45
_timestamp,1627963688
_step,0


prediction_auc,▁
_runtime,▁
_timestamp,▁
_step,▁


wandb: Agent Starting Run: 44j9egvg with config:
wandb: 	alpha: 23
wandb: 	iterations: 33
wandb: 	percent_test: 0.2001785313999015
wandb: 	regularization: 0.2717519472103404
wandb: wandb version 0.11.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


...

prediction_auc,▁
_runtime,▁
_timestamp,▁
_step,▁


wandb: Agent Starting Run: z9upult8 with config:
wandb: 	alpha: 8
wandb: 	iterations: 100
wandb: 	percent_test: 0.15990282345906398
wandb: 	regularization: 0.19311882911377312
wandb: wandb version 0.11.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


prediction_auc,0.88719
_runtime,45
_timestamp,1627988588
_step,0


prediction_auc,▁
_runtime,▁
_timestamp,▁
_step,▁


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
